In [51]:
# pip install tensorflow matplotlib scipy plotly scikit-learn
# pip install --upgrade keras
# pip install --upgrade tensorflow
# pip install scikit-learn seaborn

In [52]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from glob import glob
from keras.preprocessing import image
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications import ResNet101
from tensorflow.keras.applications import ResNet152
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import preprocess_input


In [53]:
# re-size all the images to this
image_size = [224, 224]

In [54]:
# Load the ResNet50 model without the top layer
resnet = ResNet101(weights="imagenet", include_top=False, input_shape=(224, 224, 3))

In [55]:
# Freeze base model layers to retain pre-trained features
for layer in resnet.layers:
    layer.trainable = False


In [56]:
# For getting number of output classes
folders = glob('flower_dataset/train/*')
print(len(folders))

5


In [57]:
# our layers - you can add more if you want
# x = Flatten()(resnet.output)
# prediction = Dense(len(folders), activation='softmax')(x)

# x = Flatten()(resnet.output)
# x = Dropout(0.05)(x)  # Helps prevent overfitting
# x = Dense(512, activation='relu')(x) 
# x = Dropout(0.15)(x)
# x = Dense(512, activation='relu')(x)
# x = Flatten(name="flatten2")(x)
# prediction = Dense(len(folders), activation='sigmoid')(x)  # Binary classification

model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(224,224,3)),
        data_augmentation,
        tf.keras.layers.Lambda(preprocess_input_resnet),
        base_model,
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(4, activation='softmax')
    ])

model.summary()


In [58]:
# Create the final model
model = Model(inputs=resnet.input, outputs=prediction)

In [59]:
# Compile the model
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [60]:
# model.summary()

In [61]:
# Data augmentaion on training data
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

In [62]:
validation_datagen = ImageDataGenerator(rescale=1.0/255.0)
test_datagen = ImageDataGenerator(rescale=1.0/255.0)

In [63]:
train_generator = train_datagen.flow_from_directory(
    "flower_dataset/train",  # Change this to your dataset folder path
    target_size=(224, 224),
    batch_size=8,
    class_mode="categorical"
)

Found 1760 images belonging to 5 classes.


In [64]:
val_generator = validation_datagen.flow_from_directory(
    "flower_dataset/validation",  
    target_size=(224, 224),
    batch_size=8,
    class_mode="categorical"
)


Found 438 images belonging to 5 classes.


In [65]:
test_set = validation_datagen.flow_from_directory('flower_dataset/test',
                                            target_size = (224, 224),
                                            batch_size = 8,
                                            class_mode = 'categorical')

Found 548 images belonging to 5 classes.


In [66]:
# Now time to Train the model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=50,
    verbose=1
)

Epoch 1/50
220/220 ━━━━━━━━━━━━━━━━━━━━ 90s 388ms/step - accuracy: 0.2746 - loss: 3.6465 - val_accuracy: 0.2740 - val_loss: 3.7330
Epoch 2/50
220/220 ━━━━━━━━━━━━━━━━━━━━ 85s 386ms/step - accuracy: 0.2993 - loss: 2.8905 - val_accuracy: 0.3790 - val_loss: 2.6552
Epoch 3/50
220/220 ━━━━━━━━━━━━━━━━━━━━ 85s 386ms/step - accuracy: 0.3676 - loss: 3.0242 - val_accuracy: 0.4406 - val_loss: 1.7852
Epoch 4/50
220/220 ━━━━━━━━━━━━━━━━━━━━ 85s 389ms/step - accuracy: 0.4082 - loss: 2.2606 - val_accuracy: 0.3927 - val_loss: 4.8970
Epoch 5/50
220/220 ━━━━━━━━━━━━━━━━━━━━ 87s 397ms/step - accuracy: 0.3750 - loss: 3.1498 - val_accuracy: 0.5000 - val_loss: 1.6636
Epoch 6/50
220/220 ━━━━━━━━━━━━━━━━━━━━ 85s 385ms/step - accuracy: 0.3900 - loss: 2.5883 - val_accuracy: 0.5228 - val_loss: 1.5335
Epoch 7/50
220/220 ━━━━━━━━━━━━━━━━━━━━ 85s 384ms/step - accuracy: 0.4262 - loss: 2.2753 - val_accuracy: 0.4840 - val_loss: 2.4768
Epoch 8/50
220/220 ━━━━━━━━━━━━━━━━━━━━ 84s 380ms/step - accuracy: 0.3982 - loss: 2

KeyboardInterrupt: 

In [33]:
# print(model.summary())

In [ ]:
test_loss, test_acc = model.evaluate(test_set)
print(f"Test Accuracy: {test_acc:.4f}")
print(f"Test Loss: {test_loss:.4f}")

In [ ]:
# Ploting the accuracy and loss graph

In [ ]:
print(history.history.keys())

In [ ]:
plt.figure(1, figsize = (10,4))  
plt.plot(history.history['accuracy']) 
plt.plot(history.history['val_accuracy'])  
plt.title('Model accuracy')  
plt.ylabel('accuracy')  
plt.xlabel('epoch')  
plt.legend(['train', 'valid'])

In [ ]:
plt.figure(1, figsize = (10,4)) 
plt.plot(history.history['loss'])  
plt.plot(history.history['val_loss'])  
plt.title('Model loss')  
plt.ylabel('loss')  
plt.xlabel('epoch')  
plt.legend(['train', 'valid']) 
plt.show()

In [ ]:
# Saving the model
model.save("ResNet101_multi-class_classifier.keras")

In [ ]:
# Load the model from file
from keras.models import load_model
saved_model = load_model("ResNet101_multi-class_classifier.keras")


In [ ]:
from tensorflow.keras.preprocessing import image
img1 = image.load_img('flower_dataset/testing/image_1.jpg', target_size=(224, 224))
img = image.img_to_array(img1) / 255.0
img = np.expand_dims(img, axis=0)

prediction = saved_model.predict(img)
class_idx = np.argmax(prediction, axis=1)[0]

# Get the predicted class label (assuming you have a list of class names)
class_names = ['Daisy', 'Dandelion', 'Roses', 'Sunflowers', 'Tulips'] 
predicted_class = class_names[class_idx]

# Print the predicted class with its probability
confidence = prediction[0, class_idx]
value = f"{predicted_class}: {confidence:.2f}"

# Display the image with prediction text
plt.imshow(img1)
plt.text(20, 62, value, color='red', fontsize=18, bbox=dict(facecolor='white', alpha=0.8))
plt.show()

In [42]:
import time
import sys

def fancy_print(text, delay=0.05):
    for char in text:
        sys.stdout.write(char)
        sys.stdout.flush()
        time.sleep(delay)
    print()  # New line

fancy_print("🌟 Thank you for watching this video! 🌟", 0.07)
fancy_print("👍 Please like, ❤️ share, and 🔔 subscribe! 🚀", 0.07)

🌟 Thank you for watching this video! 🌟
👍 Please like, ❤️ share, and 🔔 subscribe! 🚀
